# Classifying Common Stock as Value or Growth

## Import Packages

In [25]:
import requests
import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import mysql.connector
import config

## MySQL - Creating Database

In [28]:
# Define db_name
db_name = 'stock_data'

In [ ]:
# Connect to MySql and define cursor
cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
)
cursor = cnx.cursor()

# Create a database and close cursor/cnx
cursor.execute("CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(db_name))
cursor.close()
cnx.close()

In [31]:
# Reestablish connection and specify database
cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
        database = db_name
)
cursor = cnx.cursor()

In [139]:
# cursor.close()
# cnx.close()

cursor.execute('''DROP TABLE fundamental_data''')

In [140]:
# Create a table for the data to live
create_table_querey = """ 
CREATE TABLE fundamental_data (
     ticker varchar(6) NOT NULL primary key,
     company_name varchar(40),
     price float(10),
     shares_outstanding float(20),
     market_cap float(20),
     52_week_delta float(10),
     foward_eps float(10),
     trailing_eps float(10),
     g_revenue float(10),
     gross_margin_pct float(10),
     ebitda_margin_pct float(10),
     price_to_foward_earnings float(10),
     price_to_trailing_earnings float(10),
     price_to_book float(10),
     price_to_free_cash_flow float(10),
     debt_to_equity float(10),
     return_on_assets float(10),
     return_on_equity float(10),
     largest_institutional_owner varchar(40),
     recommendation varchar(10)
     )
 """

cursor.execute(create_table_querey)

## Make API calls to pull Stock Info

In [148]:
#create a dataframe of company details (name and ticker) to pull ticker
df = pd.read_csv('tickers.txt')
companies = df['details'].tolist()
re.findall(r"(\w*)$", companies[0])[0]
tickers = [re.findall(r"\w*$", company)[0] for company in companies]

3611

In [149]:
querystring = [] 
for ticker in tickers:
    tickers_dict = {}
    tickers_dict['symbol'] = ticker
    querystring.append(tickers_dict)

In [151]:
querystring[0]['symbol']

'PIH'

In [154]:
data = []
count = 1
for ticker in querystring[:2]:
    url = config.api_url
    params = {"region":"US","lang":"en","symbol":ticker['symbol']}
    headers = {
        'x-rapidapi-host': config.api_host,
        'x-rapidapi-key': config.api_key
        }
    response = requests.get(url, headers=headers, params=params)
    data.append(response.json())
    print(f'{count}. Added ticker: {ticker}. {len(querystring)-count} stocks remaining')
    count += 1

1. Added ticker: {'symbol': 'PIH'}. 3610 stocks remaining
2. Added ticker: {'symbol': 'FLWS'}. 3609 stocks remaining


In [162]:
data[1]['financialData']['debtToEquity']['raw']

52.765

In [160]:
fundamental_data = []

for company in data:
    fundamentals = {}
    fundamentals['ticker'] = company['symbol'],
    fundamentals['company_name'] = company['price']['shortName'],
    fundamentals['price'] = company['financialData']['currentPrice']['raw'],
    fundamentals['shares_outstanding'] = company['defaultKeyStatistics']['sharesOutstanding']['raw'],
    fundamentals['market_cap'] = company['defaultKeyStatistics']['sharesOutstanding']['raw']*company['financialData']['currentPrice']['raw'],
    fundamentals['52_week_delta'] = company['defaultKeyStatistics']['52WeekChange']['raw'],
    fundamentals['foward_eps'] = company['defaultKeyStatistics']['forwardEps']['raw'],
    fundamentals['trailing_eps'] = company['defaultKeyStatistics']['trailingEps']['raw'],
    fundamentals['g_revenue'] = company['financialData']['revenueGrowth']['raw'],
    fundamentals['gross_margin_pct'] = company['financialData']['grossMargins']['raw'],
    fundamentals['ebitda_margin_pct'] = company['financialData']['ebitdaMargins']['raw'],
    fundamentals['price_to_foward_earnings'] = round(company['financialData']['currentPrice']['raw']/company['defaultKeyStatistics']['forwardEps']['raw'],3),
    fundamentals['price_to_trailing_earnings'] = round(company['financialData']['currentPrice']['raw']/company['defaultKeyStatistics']['trailingEps']['raw'],3),
    fundamentals['price_to_book'] = company['defaultKeyStatistics']['priceToBook']['raw'],
    fundamentals['price_to_free_cash_flow'] = round(company['financialData']['currentPrice']['raw']/company['financialData']['freeCashflow']['raw'], 3),
    fundamentals['debt_to_equity'] = company['financialData']['debtToEquity']['raw'],
    fundamentals['return_on_assets'] = company['financialData']['returnOnAssets']['raw'],
    fundamentals['return_on_equity'] = company['financialData']['returnOnEquity']['raw'],
    fundamentals['largest_institutional_owner'] = company['institutionOwnership']['ownershipList'][0]['organization'],
    fundamentals['recommendation'] = company['financialData']['recommendationKey']
    fundamental_data.append(fundamentals)

KeyError: 'raw'

In [142]:
fundamental_data

{'ticker': 'NBEV',
 'company_name': 'New Age Beverages Corporation',
 'price': 2,
 'shares_outstanding': 78394000,
 'market_cap': 156788000,
 '52_week_delta': -0.6111111,
 'foward_eps': -0.16,
 'trailing_eps': -0.358,
 'g_revenue': 4.273,
 'gross_margin_pct': 0.59978,
 'ebitda_margin_pct': -0.11078,
 'price_to_foward_earnings': -12.5,
 'price_to_trailing_earnings': -5.587,
 'price_to_book': 1.0432968,
 'price_to_free_cash_flow': 0.0,
 'debt_to_equity': 43.438,
 'return_on_assets': -0.09328,
 'return_on_equity': -0.22198999,
 'largest_institutional_owner': 'Blackrock Inc.',
 'recommendation': 'buy'}

In [143]:
fundamentals_tuple = [(
    fundamentals['ticker'],
    fundamentals['company_name'],
    fundamentals['price'],
    fundamentals['shares_outstanding'],
    fundamentals['market_cap'],
    fundamentals['52_week_delta'],
    fundamentals['foward_eps'],
    fundamentals['trailing_eps'],
    fundamentals['g_revenue'],
    fundamentals['gross_margin_pct'],
    fundamentals['ebitda_margin_pct'],
    fundamentals['price_to_foward_earnings'],
    fundamentals['price_to_trailing_earnings'],
    fundamentals['price_to_book'],
    fundamentals['price_to_free_cash_flow'],
    fundamentals['debt_to_equity'],
    fundamentals['return_on_assets'],
    fundamentals['return_on_equity'],
    fundamentals['largest_institutional_owner'],
    fundamentals['recommendation']
)]

In [144]:
fundamentals_tuple

[('NBEV',
  'New Age Beverages Corporation',
  2,
  78394000,
  156788000,
  -0.6111111,
  -0.16,
  -0.358,
  4.273,
  0.59978,
  -0.11078,
  -12.5,
  -5.587,
  1.0432968,
  0.0,
  43.438,
  -0.09328,
  -0.22198999,
  'Blackrock Inc.',
  'buy')]

In [145]:
insert_query = """
INSERT INTO fundamental_data (
    ticker,
    company_name,
    price,
    shares_outstanding,
    market_cap,
    52_week_delta,
    foward_eps,
    trailing_eps,
    g_revenue,
    gross_margin_pct,
    ebitda_margin_pct,
    price_to_foward_earnings,
    price_to_trailing_earnings,
    price_to_book,
    price_to_free_cash_flow,
    debt_to_equity,
    return_on_assets,
    return_on_equity,
    largest_institutional_owner,
    recommendation
    )
VALUES (
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s,
    %s
    )
"""

In [146]:
cursor.executemany(insert_query, fundamentals_tuple)
cnx.commit()